<a href="https://colab.research.google.com/github/TheTigerHub/UTD-Summer-2025/blob/main/UTD2025_FMEA_Severity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print ("installing hugging face libraries for QLoRA")

!pip install -q accelerate==0.30.1 #hardware acceleration
print ("installed accelerate")
!pip install -q peft==0.11.1 #peft is parameter effecient fine tuning which allows LoRA and QLoRA
print ("installed peft")
!pip install -q bitsandbytes==0.43.1 #bits and bytes helps with quantization
print("installed bitsandbytes")
!pip install -q transformers==4.41.1 #hugging face library for pre-trained models, tokenizers
print("installed transformers")
!pip install -q datasets==2.19.0 #datasets is used for loading, processing, and managing datasets
print("installed datasets")
!pip install -q scikit-learn==1.4.2 # scikit-learn for data splitting
print("installed scikit-learn")
!pip install -q pandas #used for data manipulation and analysis
print("installed pandas")
!pip install -q openpyxl #read excel
print("installed openpyxl")

print ("\n libraries installed")

installing hugging face libraries for QLoRA
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 37.9 MB/s eta 0:00:00
installed accelerate
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB

In [ ]:
print ("installing hugging face libraries for QLoRA")

!pip install -q "peft[bnb]" --upgrade #installs peft pacakge with bnb extra, integrates peft and bnb

!pip install -q accelerate bitsandbytes "transformers[torch]" --upgrade #installs transformers with dependencies for PyTorch

!pip install -q datasets==2.19.0 scikit-learn==1.4.2 pandas openpyxl #installs datasets, scikit-learn, pandas, and openpyxl

print ("\n libraries installed")
print ("\n restart runtime")

installing hugging face libraries for QLoRA
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.1/411.1 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.1/362.1 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 105.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 117.4 MB/s eta 0:00:00

 libraries installed

 restart runtime


In [ ]:
#imports libraries and configures data and training

import torch #torch is necessary for the Llama model
import pandas as pd #will be used later to load, clean, and process data before converting to a hugginf face dataset
import numpy as np #library for large, multi dimensional arrays and matricies as well as operations on them
from datasets import Dataset, DatasetDict #provides effecient way to handle large datasets for machine learning, espicially natural language processing
from sklearn.model_selection import train_test_split #from sci-kit learn library, splits dataset into training and validation
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report #these provide information on the accuracy of the model
from transformers import ( #key components from transformers
    AutoTokenizer, #loads correct tokenize automatically, tokenizers convert text into numerical IDs
    AutoModelForSequenceClassification, #standard class the automatically load the correct model for sequence classification tasks
    TrainingArguments, #class to configure training
    Trainer, #class that simplifies training and eval loop for hugging face models
    DataCollatorWithPadding, #pads sequences of data in a batch to be the same legnth
    BitsAndBytesConfig # manual QLoRA config, quantization is important bc of GPU resources
)
from peft import (
    get_peft_model, #wraps model with peft adapter like QLoRA
    LoraConfig, #class for LoRA config
    TaskType, #specifies task type
    prepare_model_for_kbit_training # manual QLoRA setup with kbit quantization
)

import warnings
warnings.filterwarnings("ignore") #makes output cleaner
from huggingface_hub import notebook_login # Keep login for Llama 3.1

CSV_PATH = "2.4.xlsx" #uploaded excel file

# Column Names (Ensure these EXACTLY match your cleaned CSV/Excel headers)
COL_SUBFUNCTION = "Subfunction"
COL_REQUIREMENTS = "Requirements"
COL_FAILURE_MODE = "Potential Failure Mode and descriptions" # Base name, will be cleaned
COL_EFFECT_PRIMARY = "Potential Effect(s) of Failure (primary)" # Base name
COL_EFFECT_SECONDARY = "Potential Effect(s) of Failure (secondary)" # Base name
COL_SEVERITY = "Severity" # target column

# Input/Output Columns
INPUT_TEXT_COLS = [
    COL_SUBFUNCTION, COL_REQUIREMENTS, COL_FAILURE_MODE,
    COL_EFFECT_PRIMARY, COL_EFFECT_SECONDARY
]


#need to fill in empty cells
COLS_TO_FORWARD_FILL = [
    COL_SUBFUNCTION, COL_REQUIREMENTS, COL_FAILURE_MODE
]

#want to predict this
TARGET_COLUMN = COL_SEVERITY
NUM_LABELS = 10 # Severity can only be 1-10

# Model Config
MODEL_NAME = "meta-llama/Llama-3.2-1B" # using 3.2-1B bc 3.1 access has not been approved
MAX_SEQ_LENGTH = 512 #for memory management

# Training Config
OUTPUT_DIR = "fmea_severity_classifier_llama32_1b_standard_qlora" # <<< New output dir name
LEARNING_RATE = 1e-4      # common QLoRA starting point
NUM_EPOCHS = 3            # Train for 3 epochs which is standard for fine tuning
# MAX_STEPS = 500         # Alternative to epochs
BATCH_SIZE_PER_DEVICE = 4 # changed to 4 because model is smaller
GRAD_ACCUMULATION_STEPS = 4 # effective batch size of 16
LORA_R = 16 #the rank for LoRA matricies 16 is common
LORA_ALPHA = 32 #scaling factor for LoRA updates twice or equal to LORA_R is standard
LORA_DROPOUT = 0.05 #used for regularization, 0.05 is typical value
LOGGING_STEPS = 10 #frequent updates
SAVE_STRATEGY = "epoch" #when model should be saved
EVAL_STRATEGY = "epoch"   #when eval should be preformed
# -----done configing LoRA------


#setup device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #checks if device is uisng cuda GPU, cpu for backup
print(f"Using device: {device}") #prints out what device is being used
if device.type == 'cpu': print("Warning: Running on CPU!") #warning for if using cpu bc cpu training is way too slow

# Setup Label Mappings
labels_list = [str(i) for i in range(1, 11)] #creates list of strings for severity label from "1" to "10"
id2label = {i: label for i, label in enumerate(labels_list)} #maps integers 0-9 to strings "1"-"10" needed for classification head and output interpretation
label2id = {label: i for i, label in enumerate(labels_list)} #reverse of previous line, maps integers "1"-"10" to integers 0-9 which is required for training
print(f"id2label mapping: {id2label}") #print for verification
print(f"label2id mapping: {label2id}") #print for verification

# Check GPU capability for compute dtype in BNBConfig will  use float16 because t4 is the free avalible one
compute_dtype = torch.float16
if torch.cuda.is_available():
    if torch.cuda.get_device_capability()[0] >= 8: # Ampere+ (A100)
        compute_dtype = torch.bfloat16
        print("Compute dtype set to bfloat16 for Ampere+ GPU.")
    else:
        print("Compute dtype set to float16.")

Using device: cuda
id2label mapping: {0: '1', 1: '2', 2: '3', 3: '4', 4: '5', 5: '6', 6: '7', 7: '8', 8: '9', 9: '10'}
label2id mapping: {'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '10': 9}
Compute dtype set to float16.
